In [1]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
import h5py
from torchvision import transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
from torchvision import models
import os
import matplotlib.pyplot as plt
from PIL import Image
import math
import cv2

print(torch.__version__)
device = torch.device('cuda:0')

1.2.0


In [2]:
normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
inv_normalize = transforms.Normalize([-0.485/0.229, -0.456/0.224, -0.406/0.225], [1/0.229, 1/0.224, 1/0.225])

def tensor_to_img(t):
    """Convert normalized tensor in Cuda to cv2 image"""
    unnormalized = inv_normalize(t)
    npimg = np.transpose(unnormalized.cpu().numpy(), (1, 2, 0))
    npimg[npimg > 1] = 1
    npimg[npimg < 0] = 0
    npimg = np.uint8(np.round(npimg*255))
    return npimg

def img_to_cuda_tensor(img):
    img = img / 255.
    tr_img = np.transpose(img, (2, 0, 1))
    t = torch.from_numpy(tr_img)
    t = normalize(t.float())
    return t.to(device)
    

def imshow(img, title):
    """Custom function to display the image using matplotlib"""    
    npimg = tensor_to_img(img)
    #plot the numpy image
    plt.figure(figsize = (4, 4))
    plt.axis("off")
    plt.imshow(npimg)
    plt.title(title)
    plt.show()
    

def imshow_multi(list_img, title = None, n_cols = 5):
    n_rows = math.ceil((len(list_img))/n_cols)
    fig, axes = plt.subplots(n_rows,n_cols, figsize=(4*n_cols,4*n_rows))
    for i,ax in enumerate(axes.flat):
        ax.grid(False)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        if i>=len(list_img):
            continue
        if(title != None):
            ax.set_title("layer " + str(title[i]))

        ax.imshow(list_img[i])
        
    plt.tight_layout()

In [3]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

class My_Model(nn.Module):
    def __init__(self, input_channel=1, num_class=21, num_family = 5):
        super(My_Model, self).__init__()
        model = models.resnet18(pretrained=True)
        self.model_ft = torch.nn.Sequential(*(list(model.children())[:-1]))
        set_parameter_requires_grad(self.model_ft, False)

        self.family_fc = nn.Linear(512, num_family)
        self.class_fc = nn.Linear(512, num_class)
        
    
    def forward(self, x):
        # Perform the usual forward pass
        x = self.model_ft(x)
        x = torch.flatten(x, 1)
        x_class = self.class_fc(x)
        x_family = self.family_fc(x)
        return F.softmax(x_class, dim=1), F.softmax(x_family, dim=1)

In [4]:
_model = My_Model(num_class=21, num_family = 5)
_model.to(device)
_model.load_state_dict(torch.load('epochs/ResNet18-base-line.pt'), strict=False)

<All keys matched successfully>

In [7]:
class SaveFeatures():
    """hook function at forward step"""
    def __init__(self, module):
        self.hook = module.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output):
        self.features = output
    def close(self):
        self.hook.remove()

class ClassSpecificImageGeneration():
    """
        Produces an image that maximizes a certain class with gradient ascent
    """
    def __init__(self, model):
        self.model = model
        self.model.eval()
        self.activations = SaveFeatures(model.class_fc)
     

    def generate(self, target_class = 0, sz =128, opt_steps=160):
        rand_img = np.uint8(np.random.random((sz, sz, 3)) * 20 + 128.)
        recreated_im = []
        # Define optimizer for the image
        
        for n in range(0, opt_steps):
            rand_img = img_to_cuda_tensor(rand_img)
            opt_img = Variable(rand_img[None, :], requires_grad=True)
            optimizer = torch.optim.SGD([opt_img], lr=1)
            # Forward
            output = self.model(opt_img)
            # Target specific class
            class_loss = -self.activations.features[0, target_class] + 0.01*torch.sum(torch.abs(opt_img))
            # Backward
            self.model.zero_grad()
            class_loss.backward()
            optimizer.step()
            # Recreate image
            rand_img = tensor_to_img(opt_img.data[0])
            
            rand_img = cv2.GaussianBlur(rand_img, (3, 3), 1)
            if(n % 10 == 0):
                recreated_im.append(rand_img)
                
        return recreated_im

In [8]:
inv = ClassSpecificImageGeneration(_model)
for i in range(21):
    reconstructs = inv.generate(target_class = i)
    cv2.imwrite("weed_classes/" + str(i) + ".png", reconstructs[-1])